In [ ]:
#|default_exp inference

# Inference

> Code required for inference.

In [ ]:
#|export
from fastai.learner import Learner, load_learner
from fastcore.basics import patch

In [ ]:
#|export
@patch
def get_X_preds(self: Learner, 
    X, 
    y=None, 
    bs=64, 
    with_input=False, # returns the input as well
    with_decoded=True, # returns decoded predictions as well
    with_loss=False, # returns the loss per item as well
    act=None, # Apply activation to predictions, defaults to `self.loss_func`'s activation
    ):
    if with_loss and y is None:
        print("with_loss set to False as y is None")
        with_loss = False
    dl = self.dls.valid.new_dl(X, y=y, bs=bs)
    output = list(self.get_preds(dl=dl, with_input=with_input, with_decoded=with_decoded, with_loss=with_loss, reorder=False, act=act))
    if with_decoded and len(self.dls.tls) >= 2 and hasattr(self.dls.tls[-1], "tfms") and hasattr(self.dls.tls[-1].tfms, "decodes"):
        output[2 + with_input] = self.dls.tls[-1].tfms.decode(output[2 + with_input])
    return tuple(output)

Get the predictions and targets, optionally with_input and with_loss.

with_decoded will also return the decoded predictions (it reverses the transforms applied).

The order of the output is the following: 

- input (optional): if with_input is True
- **probabiblities** (for classification) or **predictions** (for regression)
- **target**: if y is provided. Otherwise None.
- **predictions**: predicted labels. Predictions will be decoded if with_decoded=True.
- loss (optional): if with_loss is set to True and y is not None.

In [ ]:
from tsai.data.external import get_UCR_data

In [ ]:
dsid = 'OliveOil'
X, y, splits = get_UCR_data(dsid, split_data=False)
X_test = X[splits[1]]
y_test = y[splits[1]]

In [ ]:
learn = load_learner("./models/test.pth")

⚠️ Warning: load_learner (from fastai) requires all your custom code be in the exact same place as when exporting your Learner (the main script, or the module you imported it from).

In [ ]:
test_probas, test_targets, test_preds = learn.get_X_preds(X_test, with_decoded=True)
test_probas, test_targets, test_preds

(tensor([[0.2632, 0.2575, 0.2431, 0.2362],
         [0.2632, 0.2575, 0.2431, 0.2363],
         [0.2631, 0.2575, 0.2431, 0.2363],
         [0.2631, 0.2575, 0.2431, 0.2363],
         [0.2632, 0.2575, 0.2431, 0.2363],
         [0.2632, 0.2575, 0.2431, 0.2362],
         [0.2632, 0.2575, 0.2431, 0.2362],
         [0.2631, 0.2575, 0.2432, 0.2362],
         [0.2631, 0.2575, 0.2432, 0.2362],
         [0.2632, 0.2575, 0.2431, 0.2362],
         [0.2632, 0.2575, 0.2431, 0.2362],
         [0.2631, 0.2575, 0.2431, 0.2363],
         [0.2632, 0.2575, 0.2431, 0.2362],
         [0.2631, 0.2575, 0.2432, 0.2362],
         [0.2632, 0.2575, 0.2431, 0.2362],
         [0.2630, 0.2575, 0.2432, 0.2364],
         [0.2631, 0.2574, 0.2432, 0.2363],
         [0.2632, 0.2575, 0.2431, 0.2363],
         [0.2632, 0.2575, 0.2431, 0.2362],
         [0.2632, 0.2575, 0.2431, 0.2362],
         [0.2632, 0.2575, 0.2431, 0.2362],
         [0.2632, 0.2575, 0.2431, 0.2362],
         [0.2632, 0.2575, 0.2432, 0.2362],
         [0

In [ ]:
test_probas2, test_targets2, test_preds2 = learn.get_X_preds(X_test, y_test, with_decoded=True)
test_probas2, test_targets2, test_preds2

(tensor([[0.2632, 0.2575, 0.2431, 0.2362],
         [0.2632, 0.2575, 0.2431, 0.2363],
         [0.2631, 0.2575, 0.2431, 0.2363],
         [0.2631, 0.2575, 0.2431, 0.2363],
         [0.2632, 0.2575, 0.2431, 0.2363],
         [0.2632, 0.2575, 0.2431, 0.2362],
         [0.2632, 0.2575, 0.2431, 0.2362],
         [0.2631, 0.2575, 0.2432, 0.2362],
         [0.2631, 0.2575, 0.2432, 0.2362],
         [0.2632, 0.2575, 0.2431, 0.2362],
         [0.2632, 0.2575, 0.2431, 0.2362],
         [0.2631, 0.2575, 0.2431, 0.2363],
         [0.2632, 0.2575, 0.2431, 0.2362],
         [0.2631, 0.2575, 0.2432, 0.2362],
         [0.2632, 0.2575, 0.2431, 0.2362],
         [0.2630, 0.2575, 0.2432, 0.2364],
         [0.2631, 0.2574, 0.2432, 0.2363],
         [0.2632, 0.2575, 0.2431, 0.2363],
         [0.2632, 0.2575, 0.2431, 0.2362],
         [0.2632, 0.2575, 0.2431, 0.2362],
         [0.2632, 0.2575, 0.2431, 0.2362],
         [0.2632, 0.2575, 0.2431, 0.2362],
         [0.2632, 0.2575, 0.2432, 0.2362],
         [0

In [ ]:
test_probas3, test_targets3, test_preds3, test_losses3 = learn.get_X_preds(X_test, y_test, with_loss=True, with_decoded=True)
test_probas3, test_targets3, test_preds3, test_losses3

(tensor([[0.2632, 0.2575, 0.2431, 0.2362],
         [0.2632, 0.2575, 0.2431, 0.2363],
         [0.2631, 0.2575, 0.2431, 0.2363],
         [0.2631, 0.2575, 0.2431, 0.2363],
         [0.2632, 0.2575, 0.2431, 0.2363],
         [0.2632, 0.2575, 0.2431, 0.2362],
         [0.2632, 0.2575, 0.2431, 0.2362],
         [0.2631, 0.2575, 0.2432, 0.2362],
         [0.2631, 0.2575, 0.2432, 0.2362],
         [0.2632, 0.2575, 0.2431, 0.2362],
         [0.2632, 0.2575, 0.2431, 0.2362],
         [0.2631, 0.2575, 0.2431, 0.2363],
         [0.2632, 0.2575, 0.2431, 0.2362],
         [0.2631, 0.2575, 0.2432, 0.2362],
         [0.2632, 0.2575, 0.2431, 0.2362],
         [0.2630, 0.2575, 0.2432, 0.2364],
         [0.2631, 0.2574, 0.2432, 0.2363],
         [0.2632, 0.2575, 0.2431, 0.2363],
         [0.2632, 0.2575, 0.2431, 0.2362],
         [0.2632, 0.2575, 0.2431, 0.2362],
         [0.2632, 0.2575, 0.2431, 0.2362],
         [0.2632, 0.2575, 0.2431, 0.2362],
         [0.2632, 0.2575, 0.2432, 0.2362],
         [0

In [ ]:
from fastcore.test import test_eq

In [ ]:
test_eq(test_probas, test_probas2)
test_eq(test_preds, test_preds2)
test_eq(test_probas, test_probas3)
test_eq(test_preds, test_preds3)

In [ ]:
#|eval: false
#|hide
from tsai.export import get_nb_name; nb_name = get_nb_name(locals())
from tsai.imports import create_scripts; create_scripts(nb_name)

<IPython.core.display.Javascript object>

/Users/nacho/notebooks/tsai/nbs/019_inference.ipynb saved at 2023-03-24 09:58:52
Correct notebook to script conversion! 😃
Friday 24/03/23 09:58:55 CET
